In [177]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

In [178]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [179]:
print(Base.metadata)
Base.registry

MetaData()


### Mapped classes

In [180]:
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import String, ForeignKey
from typing import Optional, List


class Cliente(Base):
    __tablename__ = 'CLIENTE'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    sobrenome: Mapped[Optional[str]]

    contas: Mapped[List['Conta']] = relationship(back_populates='cliente')

    def __repr__(self) -> str:
        return f'Cliente(id={self.id!r}, nome={self.nome!r}, sobrenome={self.sobrenome!r}, contas={self.contas!r})'


class Conta(Base):
    __tablename__ = 'CONTA'

    id: Mapped[int] = mapped_column(primary_key=True)
    saldo: Mapped[float] = mapped_column(default=0)
    id_cliente = mapped_column(ForeignKey('CLIENTE.id'))

    cliente: Mapped[Cliente] = relationship(back_populates='contas')

    def __repr__(self) -> str:
        return f'Conta(id={self.id!r}, saldo={self.saldo!r}, id_cliente={self.id_cliente!r}, cliente={self.cliente.nome!r})'

In [181]:
Base.metadata.create_all(engine)

2023-11-28 22:33:59,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:33:59,346 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CLIENTE")
2023-11-28 22:33:59,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-28 22:33:59,354 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CLIENTE")
2023-11-28 22:33:59,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-28 22:33:59,359 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CONTA")
2023-11-28 22:33:59,367 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-28 22:33:59,373 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CONTA")
2023-11-28 22:33:59,376 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-28 22:33:59,381 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CLIENTE" (
	id INTEGER NOT NULL, 
	nome VARCHAR(50) NOT NULL, 
	sobrenome VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-28 22:33:59,383 INFO sqlalchemy.engine.Engine [no key 0.00226s] ()
2023-11-28 22:33:59,389 INFO sqlalchemy.engine.Engine 
CREATE T

In [182]:
from sqlalchemy import inspect

inspect(engine).get_table_names()

2023-11-28 22:33:59,430 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:33:59,435 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-11-28 22:33:59,438 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-28 22:33:59,442 INFO sqlalchemy.engine.Engine ROLLBACK


['CLIENTE', 'CONTA']

In [183]:
alberto = Cliente(nome='alberto', sobrenome='frigatto')
alberto

Cliente(id=None, nome='alberto', sobrenome='frigatto', contas=[])

In [184]:
gustavo = Cliente(nome='gustavo')
gustavo

Cliente(id=None, nome='gustavo', sobrenome=None, contas=[])

In [185]:
from sqlalchemy.orm import Session

session = Session(engine)
session

In [186]:
session.add(alberto)
session.add(gustavo)

In [187]:
session.new

IdentitySet([Cliente(id=None, nome='alberto', sobrenome='frigatto', contas=[]), Cliente(id=None, nome='gustavo', sobrenome=None, contas=[])])

In [188]:
session.flush()

2023-11-28 22:33:59,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:33:59,732 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-28 22:33:59,736 INFO sqlalchemy.engine.Engine [generated in 0.00036s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('alberto', 'frigatto')
2023-11-28 22:33:59,740 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-28 22:33:59,745 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('gustavo', None)


In [189]:
gustavo

2023-11-28 22:33:59,793 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:33:59,799 INFO sqlalchemy.engine.Engine [generated in 0.00638s] (2,)


Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])

In [190]:
session.get(Cliente, 2)

Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])

In [191]:
alberto.id

1

In [192]:
gustavo.id

2

In [193]:
session.commit()
session.close()

2023-11-28 22:33:59,979 INFO sqlalchemy.engine.Engine COMMIT


In [194]:
from sqlalchemy import select

print(select(Cliente))

SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"


In [195]:
with Session(engine) as session:
    result = session.scalars(select(Cliente))

    for cliente in result:
        print(f'{cliente.id} -> {cliente.nome} -> {cliente.sobrenome}')

2023-11-28 22:34:00,061 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,066 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"
2023-11-28 22:34:00,069 INFO sqlalchemy.engine.Engine [generated in 0.00265s] ()
1 -> alberto -> frigatto
2 -> gustavo -> None
2023-11-28 22:34:00,075 INFO sqlalchemy.engine.Engine ROLLBACK


In [196]:
with Session(engine) as session:
    vinicius = Cliente(nome='Vinicius', sobrenome='Alcântara')
    # session.add(vinicius)

    print(vinicius in session)

False


In [197]:
with Session(engine) as session:
    print(session.execute(select(Cliente).filter_by(nome='gustavo')).scalar_one())
    print(session.scalar(select(Cliente).filter_by(nome='gustavo')))

2023-11-28 22:34:00,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,148 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-28 22:34:00,150 INFO sqlalchemy.engine.Engine [generated in 0.00234s] ('gustavo',)
2023-11-28 22:34:00,157 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:00,161 INFO sqlalchemy.engine.Engine [cached since 0.369s ago] (2,)
Cliente(id=2, nome='gustavo', sobrenome=None, contas=[])
2023-11-28 22:34:00,168 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-28 22:34:00,173 INFO sqlalchemy.engine.Engine [cached since 0.02516s ago] ('gustavo',)
2023-11-28 22:34:00,182 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CON

In [198]:
session = Session(engine)

alberto = session.get(Cliente, 1)
alberto

2023-11-28 22:34:00,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,228 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,232 INFO sqlalchemy.engine.Engine [generated in 0.00399s] (1,)
2023-11-28 22:34:00,240 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:00,242 INFO sqlalchemy.engine.Engine [cached since 0.4501s ago] (1,)


Cliente(id=1, nome='alberto', sobrenome='frigatto', contas=[])

In [199]:
alberto.nome = 'Alberto Luíz'

In [200]:
session.dirty

IdentitySet([Cliente(id=1, nome='Alberto Luíz', sobrenome='frigatto', contas=[])])

In [201]:
alberto_sobrenome = session.scalar(select(Cliente.sobrenome).where(Cliente.id == 2))
alberto_sobrenome

2023-11-28 22:34:00,349 INFO sqlalchemy.engine.Engine UPDATE "CLIENTE" SET nome=? WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,355 INFO sqlalchemy.engine.Engine [generated in 0.00689s] ('Alberto Luíz', 1)
2023-11-28 22:34:00,369 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,371 INFO sqlalchemy.engine.Engine [generated in 0.00241s] (2,)


In [202]:
session.delete(alberto)

In [203]:
session.execute(select(Cliente).filter_by(nome='Alberto Luíz')).first()

2023-11-28 22:34:00,436 INFO sqlalchemy.engine.Engine DELETE FROM "CLIENTE" WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,439 INFO sqlalchemy.engine.Engine [generated in 0.00262s] (1,)
2023-11-28 22:34:00,444 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE" 
WHERE "CLIENTE".nome = ?
2023-11-28 22:34:00,447 INFO sqlalchemy.engine.Engine [cached since 0.2985s ago] ('Alberto Luíz',)


In [204]:
fred = Cliente(nome='Fred', sobrenome='Boat')

session.add(fred)
session.commit()

2023-11-28 22:34:00,489 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?)
2023-11-28 22:34:00,494 INFO sqlalchemy.engine.Engine [generated in 0.00532s] ('Fred', 'Boat')
2023-11-28 22:34:00,499 INFO sqlalchemy.engine.Engine COMMIT


In [205]:
fred

2023-11-28 22:34:00,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,535 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,537 INFO sqlalchemy.engine.Engine [generated in 0.00155s] (3,)
2023-11-28 22:34:00,539 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:00,541 INFO sqlalchemy.engine.Engine [cached since 0.749s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [206]:
fred.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1a9a82ee140>,
 'id': 3,
 'sobrenome': 'Boat',
 'nome': 'Fred',
 'contas': []}

In [207]:
session.rollback()

2023-11-28 22:34:00,631 INFO sqlalchemy.engine.Engine ROLLBACK


In [208]:
fred.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1a9a82ee140>}

In [209]:
fred

2023-11-28 22:34:00,703 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,706 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,711 INFO sqlalchemy.engine.Engine [cached since 0.1757s ago] (3,)
2023-11-28 22:34:00,717 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:00,719 INFO sqlalchemy.engine.Engine [cached since 0.9278s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [210]:
session.delete(fred)

In [211]:
session.rollback()

2023-11-28 22:34:00,798 INFO sqlalchemy.engine.Engine ROLLBACK


In [212]:
fred

2023-11-28 22:34:00,822 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,828 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:00,831 INFO sqlalchemy.engine.Engine [cached since 0.2963s ago] (3,)
2023-11-28 22:34:00,845 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:00,848 INFO sqlalchemy.engine.Engine [cached since 1.056s ago] (3,)


Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [213]:
fred in session

True

In [214]:
session.close()

2023-11-28 22:34:00,922 INFO sqlalchemy.engine.Engine ROLLBACK


In [215]:
with Session(engine) as session:
    result = session.scalars(select(Cliente))

    for cliente in result:
        print(f'{cliente.id} -> {cliente.nome} -> {cliente.sobrenome}')

2023-11-28 22:34:00,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:00,963 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"
2023-11-28 22:34:00,966 INFO sqlalchemy.engine.Engine [cached since 0.8995s ago] ()
2 -> gustavo -> None
3 -> Fred -> Boat
2023-11-28 22:34:00,977 INFO sqlalchemy.engine.Engine ROLLBACK


In [216]:
fred

Cliente(id=3, nome='Fred', sobrenome='Boat', contas=[])

In [217]:
conta_fred = Conta(saldo=0, cliente=fred)
conta_fred

Conta(id=None, saldo=0, id_cliente=None, cliente='Fred')

In [218]:
session = Session(engine)

session.add(conta_fred)
session.commit()

2023-11-28 22:34:01,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:01,100 INFO sqlalchemy.engine.Engine INSERT INTO "CONTA" (saldo, id_cliente) VALUES (?, ?)
2023-11-28 22:34:01,103 INFO sqlalchemy.engine.Engine [generated in 0.00253s] (0.0, 3)
2023-11-28 22:34:01,106 INFO sqlalchemy.engine.Engine COMMIT


In [219]:
conta_fred

2023-11-28 22:34:01,177 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 22:34:01,182 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE "CONTA".id = ?
2023-11-28 22:34:01,185 INFO sqlalchemy.engine.Engine [generated in 0.00384s] (1,)
2023-11-28 22:34:01,190 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id AS "CLIENTE_id", "CLIENTE".nome AS "CLIENTE_nome", "CLIENTE".sobrenome AS "CLIENTE_sobrenome" 
FROM "CLIENTE" 
WHERE "CLIENTE".id = ?
2023-11-28 22:34:01,195 INFO sqlalchemy.engine.Engine [cached since 0.6595s ago] (3,)


Conta(id=1, saldo=0.0, id_cliente=3, cliente='Fred')

In [220]:
fred.contas[0]

2023-11-28 22:34:01,239 INFO sqlalchemy.engine.Engine SELECT "CONTA".id AS "CONTA_id", "CONTA".saldo AS "CONTA_saldo", "CONTA".id_cliente AS "CONTA_id_cliente" 
FROM "CONTA" 
WHERE ? = "CONTA".id_cliente
2023-11-28 22:34:01,246 INFO sqlalchemy.engine.Engine [cached since 1.454s ago] (3,)


Conta(id=1, saldo=0.0, id_cliente=3, cliente='Fred')

In [221]:
session.close()

2023-11-28 22:34:01,358 INFO sqlalchemy.engine.Engine ROLLBACK
